Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [26]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [27]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [28]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [29]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduceand tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

logistic
--------

In [5]:
batch_size = 128
beta = 0.01
graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
        + beta/2*(tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
           [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 30.226788
Minibatch accuracy: 17.2%
Validation accuracy: 16.4%
Minibatch loss at step 500: 1.604630
Minibatch accuracy: 83.6%
Validation accuracy: 79.6%
Minibatch loss at step 1000: 0.825469
Minibatch accuracy: 81.2%
Validation accuracy: 81.6%
Minibatch loss at step 1500: 0.530510
Minibatch accuracy: 83.6%
Validation accuracy: 81.5%
Minibatch loss at step 2000: 0.616086
Minibatch accuracy: 89.1%
Validation accuracy: 81.4%
Minibatch loss at step 2500: 0.761170
Minibatch accuracy: 79.7%
Validation accuracy: 81.3%
Minibatch loss at step 3000: 0.747784
Minibatch accuracy: 81.2%
Validation accuracy: 81.6%
Test accuracy: 88.8%


neural network
--------------

In [43]:
batch_size = 128
n_hidden_nodes = 1024
beta = 0.01
def NN_function(tf_dataset, variable_dict):
    weights_1 = variable_dict['weights_1']
    biases_1 = variable_dict['biases_1']
    weights_2 = variable_dict['weights_2']
    biases_2 = variable_dict['biases_2']
    logits_1 = tf.matmul(tf_dataset, weights_1) + biases_1
    input_2 = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(input_2, weights_2) + biases_2
    return logits_2

def regularisation_loss(variable_dict):
    loss = 0.0
    for matrice in variable_dict.values():
        loss += tf.nn.l2_loss(matrice)
    return loss

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights_1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, n_hidden_nodes]))
    biases_1 = tf.Variable(tf.zeros([n_hidden_nodes]))
    weights_2 = tf.Variable(
        tf.truncated_normal([n_hidden_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
  
    # Training computation.
    variable_dict = {'weights_1' : weights_1,
                     'biases_1' : biases_1,
                     'weights_2' : weights_2,
                     'biases_2' : biases_2}
    logits = NN_function(tf_train_dataset, variable_dict)
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
        + beta/2*regularisation_loss(variable_dict))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(NN_function(tf_valid_dataset, variable_dict))
    test_prediction = tf.nn.softmax(NN_function(tf_test_dataset, variable_dict))

In [37]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
           [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1900.073730
Minibatch accuracy: 10.2%
Validation accuracy: 22.5%
Minibatch loss at step 500: 127.836250
Minibatch accuracy: 85.2%
Validation accuracy: 81.6%
Minibatch loss at step 1000: 10.957674
Minibatch accuracy: 84.4%
Validation accuracy: 85.0%
Minibatch loss at step 1500: 1.334927
Minibatch accuracy: 89.8%
Validation accuracy: 85.0%
Minibatch loss at step 2000: 0.598221
Minibatch accuracy: 91.4%
Validation accuracy: 85.0%
Minibatch loss at step 2500: 0.638784
Minibatch accuracy: 85.9%
Validation accuracy: 84.3%
Minibatch loss at step 3000: 0.655260
Minibatch accuracy: 85.9%
Validation accuracy: 84.9%
Test accuracy: 91.4%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [19]:
batch_size = 32
n_hidden_nodes = 1024
beta = 0.0
def NN_function(tf_dataset, variable_dict):
    weights_1 = variable_dict['weights_1']
    biases_1 = variable_dict['biases_1']
    weights_2 = variable_dict['weights_2']
    biases_2 = variable_dict['biases_2']
    logits_1 = tf.matmul(tf_dataset, weights_1) + biases_1
    input_2 = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(input_2, weights_2) + biases_2
    return logits_2

def regularisation_loss(variable_dict):
    loss = 0.0
    for matrice in variable_dict.values():
        loss += tf.nn.l2_loss(matrice)
    return loss

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights_1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, n_hidden_nodes]))
    biases_1 = tf.Variable(tf.zeros([n_hidden_nodes]))
    weights_2 = tf.Variable(
        tf.truncated_normal([n_hidden_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
  
    # Training computation.
    variable_dict = {'weights_1' : weights_1,
                     'biases_1' : biases_1,
                     'weights_2' : weights_2,
                     'biases_2' : biases_2}
    logits = NN_function(tf_train_dataset, variable_dict)
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
        + beta/2*regularisation_loss(variable_dict))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(NN_function(tf_valid_dataset, variable_dict))
    test_prediction = tf.nn.softmax(NN_function(tf_test_dataset, variable_dict))

In [20]:
num_steps = 1001
number_of_batches = 2
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = ((step % number_of_batches) * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
           [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 200 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 345.730164
Minibatch accuracy: 6.2%
Validation accuracy: 18.3%
Minibatch loss at step 200: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 49.1%
Minibatch loss at step 400: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 49.1%
Minibatch loss at step 600: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 49.1%
Minibatch loss at step 800: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 49.1%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 49.1%
Test accuracy: 53.4%


The loss get very quikly to 0, which mean that the model cannot improve more but the validation and test are terrible 

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [20]:
batch_size = 128
n_hidden_nodes = 1024
beta = 0

def NN_function(tf_dataset, variable_dict, dropout=False):
    weights_1 = variable_dict['weights_1']
    biases_1 = variable_dict['biases_1']
    weights_2 = variable_dict['weights_2']
    biases_2 = variable_dict['biases_2']
    logits_1 = tf.matmul(tf_dataset, weights_1) + biases_1
    if dropout:
        input_2_before_dropout = tf.nn.relu(logits_1)
        input_2 = tf.nn.dropout(input_2_before_dropout, 0.5)*2
    else:
        input_2 = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(input_2, weights_2) + biases_2
    return logits_2

def regularisation_loss(variable_dict):
    loss = 0.0
    for matrice in variable_dict.values():
        loss += tf.nn.l2_loss(matrice)
    return loss

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights_1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, n_hidden_nodes]))
    biases_1 = tf.Variable(tf.zeros([n_hidden_nodes]))
    weights_2 = tf.Variable(
        tf.truncated_normal([n_hidden_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
  
    # Training computation.
    variable_dict = {'weights_1' : weights_1,
                     'biases_1' : biases_1,
                     'weights_2' : weights_2,
                     'biases_2' : biases_2}
    logits = NN_function(tf_train_dataset, variable_dict, dropout=True)
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
        + beta/2*regularisation_loss(variable_dict))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(NN_function(tf_valid_dataset, variable_dict))
    test_prediction = tf.nn.softmax(NN_function(tf_test_dataset, variable_dict))

In [21]:
num_steps = 5001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
           [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 200 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1063.995605
Minibatch accuracy: 7.0%
Validation accuracy: 29.4%
Minibatch loss at step 200: 898.384583
Minibatch accuracy: 68.0%
Validation accuracy: 72.7%
Minibatch loss at step 400: 377.820496
Minibatch accuracy: 69.5%
Validation accuracy: 75.0%
Minibatch loss at step 600: 104.330109
Minibatch accuracy: 64.1%
Validation accuracy: 71.4%
Minibatch loss at step 800: 54.240387
Minibatch accuracy: 60.9%
Validation accuracy: 70.0%
Minibatch loss at step 1000: 60.643963
Minibatch accuracy: 64.8%
Validation accuracy: 69.0%
Minibatch loss at step 1200: 40.513493
Minibatch accuracy: 63.3%
Validation accuracy: 69.2%
Minibatch loss at step 1400: 41.701534
Minibatch accuracy: 57.8%
Validation accuracy: 66.6%
Minibatch loss at step 1600: 21.944727
Minibatch accuracy: 54.7%
Validation accuracy: 67.4%
Minibatch loss at step 1800: 20.533241
Minibatch accuracy: 59.4%
Validation accuracy: 63.7%
Minibatch loss at step 2000: 9.871403
Minibatch accuracy: 62.5%
Validat

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [56]:
batch_size = 128
n_hidden_nodes1 = 1024
n_hidden_nodes2 = 256
n_hidden_nodes3 = 128
dropout = False
list_hidden_nodes = [image_size * image_size,
                     n_hidden_nodes1,
                     n_hidden_nodes2,
                     n_hidden_nodes3,
                     num_labels,]

beta = 0.001

def weights_biases_creation(list_hidden_nodes):
    variable_dict = {'weights' : [], 'biases': []}
    for i in range(len(list_hidden_nodes)-1):
        weights = tf.Variable(
            tf.truncated_normal([list_hidden_nodes[i], list_hidden_nodes[i+1]],
                                stddev=np.sqrt(2.0 / list_hidden_nodes[i])))
        biases = tf.Variable(tf.zeros([list_hidden_nodes[i+1]]))
        variable_dict['weights'].append(weights)
        variable_dict['biases'].append(biases)
    return variable_dict


def NN_function(tf_dataset, variable_dict, dropout=False):
    temp_logits = tf_dataset
    number_of_layers = len(variable_dict['weights'])
    for i in range(number_of_layers):
        weights = variable_dict['weights'][i]
        biases =  variable_dict['biases'][i]
        temp_logits = tf.matmul(temp_logits, weights) + biases
        if i < number_of_layers:
            temp_logits = tf.nn.relu(temp_logits)
            if dropout:
                temp_logits = tf.nn.dropout(temp_logits, 0.5)
    return temp_logits

def regularisation_loss(variable_dict):
    loss = 0.0
    for matrice in variable_dict['weights']:
        loss += tf.nn.l2_loss(matrice)
    for matrice in variable_dict['biases']:
        loss += tf.nn.l2_loss(matrice)
    return loss

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    variable_dict = weights_biases_creation(list_hidden_nodes)
    global_step = tf.Variable(0)
  
    # Training computation.
    logits = NN_function(tf_train_dataset, variable_dict, dropout=dropout)
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
        + beta/2*regularisation_loss(variable_dict))
  
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #learning_rate = tf.train.exponential_decay(0.5, global_step, 4000, 0.65, staircase=True)
    #optimizer = tf.train.GradientDescentOptimizer(
    #    learning_rate).minimize(
    #        loss, global_step=global_step)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(NN_function(tf_valid_dataset, variable_dict))
    test_prediction = tf.nn.softmax(NN_function(tf_test_dataset, variable_dict))

In [58]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
           [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 200 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.888466
Minibatch accuracy: 10.9%
Validation accuracy: 22.6%
Minibatch loss at step 200: 1.023196
Minibatch accuracy: 86.7%
Validation accuracy: 83.4%
Minibatch loss at step 400: 0.941021
Minibatch accuracy: 83.6%
Validation accuracy: 84.9%
Minibatch loss at step 600: 0.956567
Minibatch accuracy: 85.2%
Validation accuracy: 85.5%
Minibatch loss at step 800: 0.755972
Minibatch accuracy: 89.8%
Validation accuracy: 86.0%
Minibatch loss at step 1000: 0.824721
Minibatch accuracy: 85.2%
Validation accuracy: 86.2%
Minibatch loss at step 1200: 0.699641
Minibatch accuracy: 88.3%
Validation accuracy: 85.3%
Minibatch loss at step 1400: 0.660878
Minibatch accuracy: 91.4%
Validation accuracy: 86.5%
Minibatch loss at step 1600: 0.748600
Minibatch accuracy: 87.5%
Validation accuracy: 87.6%
Minibatch loss at step 1800: 0.669614
Minibatch accuracy: 87.5%
Validation accuracy: 87.9%
Minibatch loss at step 2000: 0.517228
Minibatch accuracy: 94.5%
Validation accuracy: 